In [ ]:
# Process to install packages not native to Google CoLab - 
# in this case we are installing PySpark & Java

# MAKE SURE TO USE THIS BLOCK OF CODE FOR EVERY NOTEBOOK THAT YOU WANT TO USE SPARK IN

import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.<enter version>'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:13 https://

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()

In [ ]:
# We can use SparkFiles to read in data directly to a DataFrame.
# This includes data from Amazon's Simple Storage Service (S3)

# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/dataviz-curriculum/day_1/wine.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("wine.csv"), sep=",", header=True)
df.show()

In [ ]:
# Now we'll deeply consider the differences between transformations and actions

In [ ]:
# Transformations

In [ ]:
# Order a DataFrame by ascending values
df.orderBy(df["points"].desc())

DataFrame[country: string, description: string, designation: string, points: string, price: string, province: string, region_1: string, region_2: string, variety: string, winery: string]

In [ ]:
# ^ This is a Transformation because we didn't ask spark to run this command,
# We only told it what we want when we do eventually call this command

# These are instructions for Spark, Spark will recognize these instructions
# but won't act on them yet because we haven't asked for it to run/produce anything

In [ ]:
# Actions

In [ ]:
# Order a DataFrame by ascending values and call/show this change
df.orderBy(df["points"].desc()).show(15)

+-------+--------------------+--------------------+------+-----+------------+--------------------+------------+--------------------+--------------------+
|country|         description|         designation|points|price|    province|            region_1|    region_2|             variety|              winery|
+-------+--------------------+--------------------+------+-----+------------+--------------------+------------+--------------------+--------------------+
| France|98-100 Barrel sam...|       Barrel sample|    99| null|    Bordeaux|             Margaux|        null|Bordeaux-style Re...|      Ch̢teau Palmer|
| France|98-100 Barrel sam...|       Barrel sample|    99| null|    Bordeaux|            Pauillac|        null|Bordeaux-style Re...|Ch̢teau Pontet-Canet|
| France|98-100 Barrel sam...|       Barrel sample|    99| null|    Bordeaux|           Sauternes|        null|Bordeaux-style Wh...|     Ch̢teau d'Yquem|
| France|A magnificent Cha...|Dom P̩rignon Oeno...|    99|  385|   Champagne

In [ ]:
# ^ This is an Action because we are telling Spark to run 
# and pull up our requested organization

# show() is an action that gives Spark the go-ahead to run 
# all of the transformations we gave it and produce a result

In [ ]:
# Spark can import more functions, such as averages for example

In [ ]:
# Import additional functions
from pyspark.sql.functions import avg
df.select(avg("points")).show()

+-----------------+
|      avg(points)|
+-----------------+
|87.88834105383143|
+-----------------+



In [ ]:
# ^ In this example the avg() function is a transformation while show() is an action

In [ ]:
# Filter data or columns
df.filter("price<20").show(8)

+--------+--------------------+--------------------+------+-----+---------------+--------------------+--------------------+------------------+--------------------+
| country|         description|         designation|points|price|       province|            region_1|            region_2|           variety|              winery|
+--------+--------------------+--------------------+------+-----+---------------+--------------------+--------------------+------------------+--------------------+
|Bulgaria|This Bulgarian Ma...|             Bergul̩|    90|   15|       Bulgaria|                null|                null|            Mavrud|        Villa Melnik|
|   Spain|Earthy plum and c...|              Amandi|    90|   17|        Galicia|       Ribeira Sacra|                null|           Menc�_a|      Don Bernardino|
|      US|There's a lot to ...|                null|    90|   18|     California|Russian River Valley|              Sonoma|        Chardonnay|            De Loach|
|      US|Massiv

In [ ]:
# ^ filter() is a transformation while show() is an action

In [ ]:
# We can Filter and select certain columns

In [ ]:
# Filter by price on certain columns
df.filter("price<25").select(['points','country','winery','price']).show(10)

+------+--------+--------------------+-----+
|points| country|              winery|price|
+------+--------+--------------------+-----+
|    95|      US|               Heitz|   24|
|    90|Bulgaria|        Villa Melnik|   15|
|    90|  France|      Bouvet-Ladubay|   22|
|    90|   Italy|    Casina di Cornia|   23|
|    90|   Spain|      Don Bernardino|   17|
|    90|      US|            De Loach|   18|
|    91|Portugal|    Herdade do Rocim|   23|
|    91|      US|   Trinity Vineyards|   19|
|    91|Portugal|Adega Cooperativa...|   15|
|    91|      US|          Eagle Glen|   22|
+------+--------+--------------------+-----+
only showing top 10 rows



In [ ]:
# ^ Filter() and select() are two separate transformations while show() is an action

In [ ]:
# Perform the same transformations using Python:

# Filter
df.filter("price<40").show(5)
# Filter by price on certain columns
df.filter("price<40").select(['points', 'country', 'winery', 'price']).show()

# Filter by exact state
df.filter(df['country'] == 'US').show()

+--------+--------------------+--------------------+------+-----+------------+--------------------+--------+---------------+--------------+
| country|         description|         designation|points|price|    province|            region_1|region_2|        variety|        winery|
+--------+--------------------+--------------------+------+-----+------------+--------------------+--------+---------------+--------------+
|      US|Heitz has made th...|          Grignolino|    95|   24|  California|         Napa Valley|    Napa|           Ros̩|         Heitz|
|Bulgaria|This Bulgarian Ma...|             Bergul̩|    90|   15|    Bulgaria|                null|    null|         Mavrud|  Villa Melnik|
|      US|Steely and perfum...|            Babushka|    90|   37|  California|Russian River Valley|  Sonoma|     Chardonnay|      Zepaltas|
|  France|Pale in color, th...|Nonpareil Tr̩sor ...|    90|   22|France Other|        Vin Mousseux|    null|Sparkling Blend|Bouvet-Ladubay|
|   Italy|Aromas of 